In [1]:
import pandas as pd
import os
import datetime
import numpy as np


def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
   

In [2]:
Weeks_2018_Q4=[datetime.date(2018,11,10)+datetime.timedelta(days=x*7) for x in range(13)]

Q4_2018_weekly_data=list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/"))
Q4_2018_weekly_data=[x for x in Q4_2018_weekly_data if ("SalesWeekly" in x)]

Q4_2019_weekly_data=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
Q4_2019_weekly_data=[x for x in Q4_2019_weekly_data if ("SalesWeekly" in x)]


Q4_weekly_data_list=[]
for week in Weeks_2018_Q4:
    weekly_file=[x for x in Q4_2018_weekly_data+Q4_2019_weekly_data if str(week) in x]
    if len(weekly_file)==1:
        Q4_weekly_data_list=Q4_weekly_data_list+weekly_file


In [3]:
df_output=pd.DataFrame()
df_by_store=pd.DataFrame()
for file in Q4_weekly_data_list:
    try:
        df=pd.read_table(file,dtype=str,sep="|",usecols=['location_id','week_end_dt','gross_transaction_cnt','subclass_gross_sales_amt'])
        df['subclass_gross_sales_amt']=df['subclass_gross_sales_amt'].astype(float)
        df['gross_transaction_cnt']=df['gross_transaction_cnt'].astype(int)
        df=df.rename(columns={"subclass_gross_sales_amt":"sales",'gross_transaction_cnt':"transactions"})
        df['type']=np.where(df['location_id']=="6990","Ecommerce","InStore")
        
        df_sales=df.groupby(['location_id','week_end_dt','type'])['sales'].sum().to_frame().reset_index()
        df_trans=df[['location_id','week_end_dt','type','transactions']].drop_duplicates()
        df=pd.merge(df_sales,df_trans,on=["location_id",'week_end_dt','type'],how="outer")
        
        df_by_store=df_by_store.append(df)
        df=df.groupby(['type','week_end_dt'])['sales','transactions'].sum().reset_index()
        df_output=df_output.append(df)
    except:
        df=pd.read_table(file,dtype=str,sep="|",usecols=['location_id','week_end_dt','gross_transaction_cnt','class_gross_sales_amt'])
        df['class_gross_sales_amt']=df['class_gross_sales_amt'].astype(float)
        df['gross_transaction_cnt']=df['gross_transaction_cnt'].astype(int)
        df=df.rename(columns={"class_gross_sales_amt":"sales",'gross_transaction_cnt':"transactions"})
        df['type']=np.where(df['location_id']=="6990","Ecommerce","InStore")
        
        df_sales=df.groupby(['location_id','week_end_dt','type'])['sales'].sum().to_frame().reset_index()
        df_trans=df[['location_id','week_end_dt','type','transactions']].drop_duplicates()
        df=pd.merge(df_sales,df_trans,on=["location_id",'week_end_dt','type'],how="outer")
        
        df_by_store=df_by_store.append(df)
        df=df.groupby(['type','week_end_dt'])['sales','transactions'].sum().reset_index()
        df_output=df_output.append(df)

In [4]:
df.head(2)

,type,week_end_dt,sales,transactions
0,Ecommerce,2019-02-02,304629.13,3295
1,InStore,2019-02-02,73125938.18,2400351


In [5]:
type_df=df_output.groupby(['type'])['sales','transactions'].sum().reset_index()

In [6]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Q2_2018_Sales_Ecom_InStore_Bruce_0320/BL_2018Q4_total_Sales_Trans.xlsx",engine="xlsxwriter")
type_df.to_excel(writer,"Total_Type",index=False)
df_output.to_excel(writer,"Total_Type_week",index=False)
df_by_store.to_excel(writer,"Store_Week",index=False)
writer.save()